### Introduction
#### A brief analysis of the churn rate of the telecomunications **Telco Company** ([Link to data](https://www.kaggle.com/blastchar/telco-customer-churn)) using the **CRISP-DM** process: 
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

Main objectives: 
- Better understand the CRISP-DM process, specifically focusing on:
    - Investigating the dataset & setting up business questions
    - Preparing & cleaning up data 
    - Modeling & evaluating the dataset
- Become familiarized with simple churn analysis

In [ ]:
# Import modules
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model # LinearRegression
from sklearn import model_selection # train_test_split
from sklearn import metrics # r2_score, mean_squared_error
# Import required modules
from sklearn import preprocessing

from IPython.display import display
%matplotlib inline

cwd = os.getcwd()
ls = os.listdir()
print(cwd)
print(ls)

### Import data

In [ ]:
# Import
df = pd.read_csv('.\data\WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Dataset overview
df.head()


In [ ]:
# Rows & columns
rows = df.shape[0]
cols = df.shape[1]
print(f'Number of rows = {rows} & number of columns = {cols}')

### Business questions to be analyzed in notebook
1. What are the total & monthly charges of current vs churned customers and is there a correlation between charges & churning? 
2. What is the tenure of current vs churned customers? How does tenure correlate with monthly / total charges (tornado plot)?
3. What services do people that are customers prefer vs people that churn? (nice tornado plot?)
4. Can we predict the churn rate based on the quantitative variables & how does the prediction improve when we add the categorical variables?

In [ ]:
# Check dtypes of variables
df.dtypes.to_frame().T

In [ ]:
df["TotalCharges"].head()

In [ ]:
# # --> Will give an ERROR!!!!
# Convert TotalCharges column to numeric variable 
pd.to_numeric(df['TotalCharges'], errors = 'raise')

#### Clean up table

In [ ]:
# Number of null or empty strings per variable
# Clean up data-set
# Nans
total_null_count = 0
index_null_values = []

#Check how many empty & blank strings in table
df_objects = df.select_dtypes(include=['object']).copy()
for col in df.columns:
    # print(col)
    if col in df_objects.columns:
        # Search for empty strings & blank with regex expression r"^\s*$" in categorical variables
        col_null_count = df[df[col].str.contains(r"^\s*$")==True][col].count()
        col_index_null_values = df[df[col].str.contains(r"^\s*$")==True].index.tolist()
        # print(str(col) + "\t" + str(df[col].dtypes) + "\t" + str(col_null_count))
        total_null_count += col_null_count
        index_null_values.extend(col_index_null_values)
        
    else: 
        # search for NaNs in numeric variabls
        col_null_count = df[col].isna().sum()
        col_index_null_values = df[np.isnan(df[col])].index.tolist()
        # print(str(col) + "\t" + str(df[col].dtypes) + "\t" + str(col_null_count))
        total_null_count += col_null_count
        index_null_values.extend(col_index_null_values)

print(f'Number of nan or empty values in table = {total_null_count}')
print(f'Nan or empty table rows: {index_null_values}')


In [ ]:
# Clean rows that contain Nan or empty values (in total charges)
df_clean = df.drop(index_null_values, inplace=False).copy()
df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors = 'raise')

# pd.to_numeric(df_clean['TotalCharges'], errors = 'raise')


rows_clean = df_clean.shape[0]
cols_clean = df_clean.shape[1]

print(f'New number of rows = {rows_clean} vs old number of rows = {rows}')
print(f'New number of cols = {cols_clean} vs old number of cols = {cols}')

In [ ]:
current_customer = df_clean[df_clean.Churn == 'No'] # Subset df_clean to only those that are currently customers
churned_customer = df_clean[df_clean.Churn == 'Yes'] # Subset df_clean to only those that have churned

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(list(pd.unique(df_clean["Churn"])),[current_customer.shape[0], churned_customer.shape[0]], label=[current_customer.shape[0], churned_customer.shape[0]])
ax.legend()
ax.set_xlabel('Churned'), ax.set_ylabel('Count'), ax.set_title('Count of churned customers')
plt.show()



# Analyze continuous data

##### What are the statistics of monthly & total charges of customers vs people that churn?

In [ ]:
sns.heatmap(df_clean.corr(), annot=True, fmt='.2f')

### Total charges

In [ ]:
total_charges_customer = df_clean[df_clean.Churn == 'No']['TotalCharges']
total_charges_churned = df_clean[df_clean.Churn == 'Yes']['TotalCharges']

print(f'Unique values in churned variable: {pd.unique(df_clean["Churn"])}')
print(f'Number of customers: {total_charges_customer.shape[0]}')
print(f'Number churned: {total_charges_churned.shape[0]}')

In [ ]:
quantile_labels = total_charges_customer.describe()[-4:-1].index
quantiles_customer = total_charges_customer.describe()[-4:-1]
quantiles_churned = total_charges_churned.describe()[-4:-1]

# Create a table for quantiles & associated values
quantiles_customer_and_churned_total = pd.concat([quantiles_customer, quantiles_churned], axis=1)
ratio_total = quantiles_customer / quantiles_churned
quantiles_customer_and_churned_total = pd.concat([quantiles_customer_and_churned_total, ratio_total], axis=1)
quantiles_customer_and_churned_total.columns = ['Customers', 'Churned', 'Customer vs Churned ratio'] 
quantiles_customer_and_churned_total = quantiles_customer_and_churned_total.style.set_caption("Quantiles of total charges of currnet & churned customers")
quantiles_customer_and_churned_total 

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))
# PLOTS
# Histogram
# total_charges_customer.plot(kind = "hist", color = 'grey', alpha=0.5, density = True, bins = 15, label='customers', ax=ax) # change density to true, because KDE uses density
# total_charges_churned.plot(kind = "hist", color = 'salmon', alpha=0.5, density = True, bins = 15, label='churned', ax=ax) # change density to true, because KDE uses density

# KDE
total_charges_customer.plot(kind = "kde", color='black', label='customers', ax=ax)
total_charges_churned.plot(kind = "kde", color='red', label='churned', ax=ax)

# Plot percentiles
# total_charges_customer.plot(kind = "kde", color='black', label='TotalCharges')
# total_charges_churned.plot(kind = "kde", color='red', label='TotalCharges')
trans = ax.get_xaxis_transform()

for i in range(3):
    ax.axvline(quantiles_customer[i], color='black', ymax=0.5, alpha = 1, linestyle = ":")
    ax.axvline(total_charges_churned.describe()[-4:-1][i], color='red', ymax=0.75, alpha = 1, linestyle = ":")
    plt.text(quantiles_customer[i], .52, str(quantiles_customer[i]) + '\n(' + str(quantile_labels[i]) + ')', color = 'black', transform=trans, ha='center', va='bottom')
    plt.text(quantiles_churned[i], .77, str(total_charges_churned.describe()[-4:-1][i]) + '\n(' + str(quantile_labels[i]) + ')', color = 'red', transform=trans, ha='center', va='bottom')

# X & Y labels & title
ax.set_xlabel("Charges ($)"), ax.set_ylabel('Frequency'), ax.set_title("Total charges of current vs churned customers")
# Configure plot
ax.set_xlim(0, 10000)
ax.set_ylim(0, 0.001)
# ax.tick_params(left = False, bottom = False)
ax.set_yticks([])
ax.spines[['left', 'right', 'top']].set_visible(False)
ax.legend()

> This result shows that the **current customers have higher total charges than the churned customers**, with the P50 being ~2 times larger for the current customers in comparison to the churned customers. It would be worthwhile to investigate what causes them to have higher total charges. 1st hypothesis that comes to mind is having a longer tenure. If so, it would be important to answer what makes them stay longer?

### Monthly charges

In [ ]:
# filter data
# ALL customers - split data set into customers vs churned 
monthly_charges_customer = df_clean[df_clean.Churn == 'No']['MonthlyCharges']
monthly_charges_churned = df_clean[df_clean.Churn == 'Yes']['MonthlyCharges']

print(f'Number of customers: {monthly_charges_customer.shape[0]}')
print(f'Number churned customers: {monthly_charges_churned.shape[0]}')

# OLD cusomers - split data set into old customers vs old churned (without new customers where TotalCharges = 0)
monthly_charges_customer_old = df_clean[(df_clean.Churn == 'No') & (df_clean.TotalCharges != 0)]['MonthlyCharges'] 
monthly_charges_churned_old = df_clean[(df_clean.Churn == 'Yes') & (df_clean.TotalCharges != 0)]['MonthlyCharges']

# print(f'Number of old customers: {monthly_charges_customer_old.shape[0]}')
# print(f'Number of old churned customers: {monthly_charges_churned_old.shape[0]}')

print(f'Number of old customers (where total charges > 0) = total number of customers: {monthly_charges_customer_old.shape[0] == monthly_charges_customer.shape[0]}')

# TODO:
# --> plot histograms as line plot: https://towardsdatascience.com/take-your-histograms-to-the-next-level-using-matplotlib-5f093ad7b9d3
# --> compare average / p10 / p90 percentiles + skeweness
# --> What if we take away the customers that just joined (i.e. have no total charges)?
# --> Is there a clear difference in the monthly & total charges between people that stay vs people that churned? 

In [ ]:
quantile_labels = monthly_charges_customer.describe()[-4:-1].index
quantiles_customer_monthly = monthly_charges_customer.describe()[-4:-1]
quantiles_churned_monthly = monthly_charges_churned.describe()[-4:-1]

# Compare statistics of all vs old customers
ratio_all_monthly = quantiles_customer_monthly / quantiles_churned_monthly
quantiles_customer_and_churned_monthly = pd.concat([quantiles_customer_monthly, quantiles_churned_monthly, ratio_all_monthly], axis=1)
quantiles_customer_and_churned_monthly.columns = ['Customers', 'Churned', 'Customers vs churned ratio'] 
quantiles_customer_and_churned_monthly = quantiles_customer_and_churned_monthly.style.set_caption("Quantiles of monthly charges of current & churned customers")
quantiles_customer_and_churned_monthly 

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

# FIGURE 1
# PLOTS
# KDE
monthly_charges_customer.plot(kind = "kde", color='black', label='Customers', ax=ax)
monthly_charges_churned.plot(kind = "kde", color='red', label='Churned', ax=ax)

# Plot percentiles
# monthly_charges_customer.plot(kind = "kde", color='black', label='MonthlyCharges')
# monthly_charges_churned.plot(kind = "kde", color='red', label='MonthlyCharges')
trans = ax.get_xaxis_transform()

for i in range(3):
    ax.axvline(quantiles_customer_monthly[i], color='black', ymax=0.5, alpha = 1, linestyle = ":")
    ax.axvline(quantiles_churned_monthly[i], color='red', ymax=0.75, alpha = 1, linestyle = ":")
    plt.text(quantiles_customer_monthly[i], .52, str(quantiles_customer_monthly[i]) + '\n(' + str(quantile_labels[i]) + ')', color = 'black', transform=trans, ha='center', va='bottom')
    plt.text(quantiles_churned_monthly[i], .77, str(quantiles_churned_monthly[i]) + '\n(' + str(quantile_labels[i])+ ')', color = 'red', transform=trans, ha='center', va='bottom')

# X & Y labels & title
ax.set_xlabel("Charges ($)"), ax.set_ylabel('Frequency'), ax.set_title("Monthly charges of current vs churned customers")
# Configure plot
ax.set_xlim(0, 140), ax.set_ylim(0, 0.05)
# ax.tick_params(left = False, bottom = False)
ax.set_yticks([])
ax.spines[['left', 'right', 'top']].set_visible(False)
ax.legend()
# plt.show()

> This result shows that the **current customers have lower monthly charges than the churned customers**, with the P25/P50/P75 all being lower for the current customers in comparison to the churned customers. It is interesting that while the total charges for current customers are higher, the monthly charges for the current customers are lower. One reasonable explanation for this would be a longer tenure for current customers that have lower monthly charges. Moreover, after 60$ the churn rate seems to increase substantially. 

In [ ]:
# Create series of tenure of current & churned customers
tenure_customer = df_clean[df_clean.Churn == 'No']['tenure']#.value_counts(normalize=True, bins = [0.001,1,2,3,4,5,10,15,20,30,40,50,60], sort=False)
tenure_churned = df_clean[df_clean.Churn == 'Yes']['tenure']#.value_counts(normalize=True, bins = [0.001,1,2,3,4,5,10,15,20,30,40,50,60], sort=False)


In [ ]:
def merged_variable_statistics_current_vs_churned(current, churned, columns = ['Current', 'Churned', 'Current vs churned ratio'], title='Title'):
    #Get quantiles
    quantile_labels = current.describe()[-4:-1].index
    quantiles_current = current.describe()[-4:-1]
    quantiles_churned = churned.describe()[-4:-1]
    # Compare statistics of all vs old customers
    ratio = quantiles_current / quantiles_churned
    quantile_values = pd.concat([quantiles_current, quantiles_churned, ratio], axis=1)
    quantile_values.columns = columns
    quantiles_current_and_churned = quantile_values.style.set_caption(title)
    display(quantiles_current_and_churned)
    return quantile_labels, quantile_values

In [ ]:
quantile_labels, quantile_values = merged_variable_statistics_current_vs_churned(tenure_customer, tenure_churned, title='Quantiles of tenure of current & churned customers')

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))
# FIGURE 1
# PLOTS
# KDE
tenure_customer.plot(kind = "kde", color='black', label='Customers', ax=ax)
tenure_churned.plot(kind = "kde", color='red', label='Churned', ax=ax)

# # Plot percentiles
trans = ax.get_xaxis_transform()
ax.axvline(quantile_values['Current']['50%'], color='black', ymax=0.5, alpha = 1, linestyle = ":")
ax.axvline(quantile_values['Churned']['50%'], color='red', ymax=0.75, alpha = 1, linestyle = ":")
plt.text(quantile_values['Current']['50%'], .52, str(quantile_values['Current']['50%']) +  '\n(' + str(quantile_labels[1]) + ')', color = 'black', transform=trans, ha='center', va='bottom')
plt.text(quantile_values['Churned']['50%'], .77, str(quantile_values['Churned']['50%']) +  '\n(' + str(quantile_labels[1]) + ')', color = 'red', transform=trans, ha='center', va='bottom')

# X & Y labels & title
ax.set_xlabel("Tenure (Months)"), ax.set_ylabel('Frequency'), ax.set_title("Tenure of current vs churned customers")
# Configure plot
ax.set_xlim(0, 90), ax.set_ylim(0, 0.05)
ax.set_yticks([])
ax.spines[['left', 'right', 'top']].set_visible(False)
ax.legend()

> This result shows that the **current customers have longer tenure than the churned customers**, with the P25/P50/P75 all being higher for the current customers in comparison to the churned customers. This is as expected, since the current customers had larger total charges, while maintaining lower monthly charges. 

# Analyze categorical data

In [ ]:
print("Categorical data: ", list(df_clean.select_dtypes(include=['object']).columns))


In [ ]:
categories = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 
'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
'Contract', 'PaperlessBilling', 'PaymentMethod']

In [ ]:
print("Categories")
for col in df_clean.columns:
       if col in categories:
              print(col, '\t', df_clean[col].unique(), '\t length: ', len(df_clean[col].unique()))

In [ ]:
# floor()
int(np.ceil(len(categories) / 3))


In [ ]:
def categorical_analysis_plots(df_clean, categories):
    # fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,5))

    fig, axes = plt.subplots(int(np.ceil(len(categories) / 3)), 3, figsize=(16, 16))
    fig.suptitle('Categorical data')
    i_idx = 0
    j_idx = 0
    total_idx = 0
    for cat in categories:
        # print(f'i = {i_idx}, j = {j_idx}')
        # ax = sns.catplot(ax = axes[i_idx, j_idx], x=cat, hue="Churn", kind="count", legend_out=False, data=df_clean).set(title=f'Current vs churned customers based on {cat}')
        sns.countplot(ax = axes[i_idx, j_idx], x=cat, hue="Churn", data=df_clean).set(title=f'{cat}')
        if len(df_clean[cat].unique()) > 3:
            axes[i_idx, j_idx].set_xticklabels(axes[i_idx, j_idx].get_xticklabels(), rotation=40, ha="right")
        # axes[i_idx, j_idx].set_xticklabels(axes[i_idx, j_idx].get_xticklabels(), rotation=40, ha="right")
        #  = sns.countplot(x="Column", data=ds)

        # _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
        # if len(df_clean[cat].unique()) > 2:
        #     locs, labels = plt.xticks()
        #     plt.setp(labels, rotation=45)
        # plt.legend(loc="upper right")
        j_idx += 1
        if j_idx > 2:
            j_idx = 0
            i_idx += 1
    plt.tight_layout()

In [ ]:
categorical_analysis_plots(df_clean, categories)

In [ ]:
from collections import defaultdict

# Function taken from UDACITY data science class
def total_count(df, col1, col2, look_for):
    new_df = defaultdict(int)
    #loop through list of ed types
    for val in look_for:
        #loop through rows
        for idx in range(df.shape[0]):
            #if the ed type is in the row add 1
            if val in df[col1][idx]:
                new_df[val] += int(df[col2][idx])
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = [col1, col2]
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df


possible_vals = list(df_clean['Contract'].unique())

def clean_and_plot(df, title='Method of Educating Suggested', plot=True):
    # Plot bar graph of different contracts (for current & churned customers)
    x = df['Contract'].value_counts().reset_index()
    x.rename(columns={'index': 'contract', 'Contract': 'count'}, inplace=True)
    x_df = total_count(x, 'contract', 'count', possible_vals)

    x_df.set_index('contract', inplace=True)
    if plot:
        (x_df/x_df.sum()).plot(kind='bar', legend=None)
        plt.title('Contract type')
        plt.show()
    props_x_df = x_df/x_df.sum()
    return props_x_df

In [ ]:
current_customer_perc = clean_and_plot(current_customer, 'Customer', plot=False)
churned_customer_perc = clean_and_plot(churned_customer, 'Churned', plot=False)

comp_df = pd.merge(current_customer_perc, churned_customer_perc, left_index=True, right_index=True)
comp_df.columns = ['current_customer_perc', 'churned_customer_perc']
comp_df['Diff_Contract_Type'] = comp_df['current_customer_perc'] - comp_df['churned_customer_perc']
comp_df.style.bar(subset=['Diff_Contract_Type'], align='mid', color=['#d65f5f', '#5fba7d'])

> **Month-to-month** contracts are the most popular type of contracts both for current and churned customers. For customers that churned, more than 88% of their contracts are month-to-month, which is compared to c. 43% of current customers which are currently on that type of contract. 

> **Long-term contracts** that are equal to or are longer than a year account for a little over 10% of customers that have churned, whereas for current customers, long-term contracts account for over 55% of all of their contracts. 

> It appears that customers with short-term month-to-month contracts are generally more likely to churn.

In [ ]:
# Leave only categorical data

obj_df = df_clean.select_dtypes(include=['object']).copy()
new_obj_df = obj_df.drop(columns = 'customerID')


In [ ]:
def create_dummy_df(df, cat_cols, dummy_na): 
    for col in  cat_cols: 
        try: 
            # for each cat add dummy var, drop original column 
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1) 
        except: 
            continue 
    return df

In [ ]:
obj_cols_lst = new_obj_df.columns 
df_cat = create_dummy_df(new_obj_df, obj_cols_lst, dummy_na=False)
print(f'shape of df_cat: {df_cat.shape}')
print(f'columns: {df_cat.columns}')
df_cat.head()

In [ ]:
sns.heatmap(df_cat.corr(), fmt='.2f')

# Train prediction model
# Linear Model

In [ ]:
def fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size, rand_state=42):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    
    The function does:
    1. Split your data into an X matrix and a response vector y
    2. Create training and test sets of data
    3. Instantiate a LinearRegression model with normalized data
    3. Fit your model to the training data
    4. Predict the response for the training data and the test data
    5. Obtain an rsquared value for both the training and test data
    '''
    
    X = df.drop(response_col, axis=1) #5 
    y = df[response_col]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = test_size, random_state=rand_state) #6
    lm_model = linear_model.LinearRegression(normalize=True) #7
    lm_model.fit(X_train, y_train) #8
    y_test_preds = lm_model.predict(X_test) #9
    y_train_preds = lm_model.predict(X_train)
    test_score = metrics.r2_score(y_test, y_test_preds) #10
    train_score = metrics.r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

In [ ]:
test_score_cat_linear_mod, train_score_cat_linear_mod, lm_model_cat, X_train_cat, X_test_cat, y_train_cat, y_test_cat = fit_linear_mod(
    df_cat, 'Churn_Yes', obj_cols_lst, dummy_na=False, test_size=.2, rand_state=42)

In [ ]:
print(f'Using only categorical data, we get a train r2 score of: \n{train_score_cat_linear_mod} \nand a test r2 score of: \n{test_score_cat_linear_mod}')

# Normalize - continuous variables

In [ ]:
#Normalize a column
def normalize_cont_columns(df):
    df_num = df.select_dtypes(exclude=['object'])
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()

    for col in df_num:
        df_num[col] = min_max_scaler.fit_transform(df_num[[col]].values.astype(float))

    return df_num

In [ ]:
df_num = normalize_cont_columns(df_clean)

In [ ]:
# df_cat_and_num = pd.concat([df_cat, df_clean.select_dtypes(exclude=['object'])], axis=1)
df_cat_and_num = pd.concat([df_cat, df_num], axis=1)
df_cat_and_num.head()

In [ ]:
df_cat_and_num.corr()['Churn_Yes'].sort_values(ascending=False).plot(kind='bar', figsize=(20,5))

In [ ]:
test_score_norm_linear_mod, train_score_norm_linear_mod, lm_model, X_train, X_test, y_train, y_test = fit_linear_mod(
    df_cat_and_num, 'Churn_Yes', obj_cols_lst, dummy_na=False, test_size=.2, rand_state=42)

In [ ]:
print(f'Using both categorical & continuous data, we get a train r2 score of: \n{train_score_norm_linear_mod} vs {train_score_cat_linear_mod} \nand a test r2 score of: \n{test_score_norm_linear_mod} vs {test_score_cat_linear_mod}')

In [ ]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = coefficients
    coefs_df['abs_coefs'] = np.abs(coefficients)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df#.head(20)

# Logistic regression

In [ ]:
def feature_weights(X_df, classifier, classifier_name):
    weights = pd.Series(classifier.coef_[0], index=X_df.columns.values).sort_values(ascending=False)

    top_weights_selected = weights[:10]
    plt.figure(figsize=(7,6))
    plt.tick_params(labelsize=10)
    plt.title(f'{classifier_name} - Top 10 Features')
    top_weights_selected.plot(kind='bar')
    
    bottom_weights_selected = weights[-10:]
    plt.figure(figsize=(7,6))
    plt.tick_params(labelsize=10)
    plt.title(f'{classifier_name} - Bottom 10 Features')
    bottom_weights_selected.plot(kind='bar')

    return print("")

def confusion_matrix_plot(X_train, y_train, X_test, y_test, classifier, y_pred, classifier_name):
    fig, ax = plt.subplots(figsize=(7, 6))
    metrics.plot_confusion_matrix(classifier, X_test, y_test, display_labels=["No Churn", "Churn"], cmap=plt.cm.Blues, normalize=None, ax=ax)
    ax.set_title(f'{classifier_name} - Confusion Matrix')
    plt.show()

    fig, ax = plt.subplots(figsize=(7, 6))
    metrics.plot_confusion_matrix(classifier, X_test, y_test, display_labels=["No Churn", "Churn"], cmap=plt.cm.Blues, normalize='true', ax=ax)
    ax.set_title(f'{classifier_name} - Confusion Matrix (norm.)')
    plt.show()

    print(f'Accuracy score test: {metrics.accuracy_score(y_test, y_pred)}')
    print(f'Accuracy score train: {classifier.score(X_train, y_train)} (as comparison)')
    return print("")

def roc_curve_auc_score(X_test, y_test, y_pred_probabilities, classifier_name):

    y_pred_prob = y_pred_probabilities[:, 1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)

    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label=f'{classifier_name}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{classifier_name} - ROC Curve')
    plt.show()

    return print(f'AUC Score (ROC): {metrics.roc_auc_score(y_test, y_pred_prob)}\n')

def precision_recall_curve_and_scores(X_test, y_test, y_pred, y_pred_probabilities, classifier_name):
    y_pred_prob = y_pred_probabilities[:, 1]
    precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)

    plt.plot(recall, precision, label=f'{classifier_name}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'{classifier_name} - Precision-Recall Curve')
    plt.show()

    f1_score_result, auc_score_result = metrics.f1_score(y_test, y_pred), metrics.auc(recall, precision)

    return print(f'f1 Score: {f1_score_result} \n AUC Score (PR): {auc_score_result}\n')
    

In [ ]:
def fit_logreg_mod(df, response_col, test_size, rand_state=42):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    
    The function does:
    1. Split your data into an X matrix and a response vector y
    2. Create training and test sets of data
    3. Instantiate a LinearRegression model with normalized data
    3. Fit your model to the training data
    4. Predict the response for the training data and the test data
    5. Obtain an rsquared value for both the training and test data
    '''
    
    X = df.drop(response_col, axis=1) #5 
    y = df[response_col]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = test_size, random_state=rand_state) #6
    lm_model = linear_model.LogisticRegression(random_state=rand_state, max_iter = 100) #7
    lm_model.fit(X_train, y_train) #8
    y_test_preds = lm_model.predict(X_test) #9
    y_train_preds = lm_model.predict(X_train)
    test_score = metrics.r2_score(y_test, y_test_preds) #10
    train_score = metrics.r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

In [ ]:
test_score_logreg_mod, train_score_logreg_mod, lm_model_logreg, X_train_logreg, X_test_logreg, y_train_logreg, y_test_logreg = fit_logreg_mod(
    df_cat_and_num, 'Churn_Yes', .2, 42)

In [ ]:
print(f'Using linear model we get a train r2 score of: \n{train_score_norm_linear_mod} vs logistic regression model {train_score_logreg_mod} \nand a test r2 score of: \n{test_score_norm_linear_mod} vs {test_score_logreg_mod}')

In [ ]:
y_pred_logreg = lm_model_logreg.predict(X_test)
y_pred_logreg_prob = lm_model_logreg.predict_proba(X_test)

In [ ]:
X1 = df_cat_and_num.drop('Churn_Yes', axis=1)

feature_weights(X1, lm_model_logreg, 'Log. Regression')
confusion_matrix_plot(X_train_logreg, y_train_logreg, X_test_logreg, y_test_logreg, lm_model_logreg, y_pred_logreg, 'Log. Regression')
roc_curve_auc_score(X_test_logreg, y_test_logreg, y_pred_logreg_prob, 'Log. Regression')
precision_recall_curve_and_scores(X_test_logreg, y_test_logreg, y_pred_logreg, y_pred_logreg_prob, 'Log. Regression')

In [ ]:
#Use the function
coef_df = coef_weights(lm_model_logreg.coef_[0], X_train_logreg)

#A quick look at the top results
coef_df#.head(20)

In [ ]:
from sklearn import neighbors

def fit_knn_mod(df, response_col, test_size=.3, rand_state=42):    
    X = df.drop(response_col, axis=1) #5 
    y = df[response_col]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = test_size, random_state=rand_state) #6
    lm_model = neighbors.KNeighborsClassifier() #random_state=rand_state, max_iter = 1000) #7
    lm_model.fit(X_train, y_train) #8
    y_test_preds = lm_model.predict(X_test) #9
    y_train_preds = lm_model.predict(X_train)
    test_score = metrics.r2_score(y_test, y_test_preds) #10
    train_score = metrics.r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test, y_test_preds, y_train_preds

In [ ]:
test_score_knn, train_score_knn, lm_model_knn, X_train_knn, X_test_knn, y_train_knn, y_test_knn, y_test_preds_knn, y_train_preds_knn = fit_knn_mod(
    df_cat_and_num, 'Churn_Yes', test_size=.3, rand_state=42)

print(f'Using linear model we get a train r2 score of: \n{train_score_knn} \nand a test r2 score of: \n{test_score_knn}')
# from sklearn 
